# MODELO TIME SERIES ORDER SERIES

Cargar datos y disponibilidad de datos en fechas

In [ ]:
import pandas as pd

# Cargar el archivo
file_path = './data/orders_orders.xlsx'
df = pd.read_excel(file_path)

# Ver rango de fechas
print("Rango de fechas:", df['Order Date'].min(), "a", df['Order Date'].max())

Sumar cifra diaria y escoger las columnas que utilizaremos para el timeseries modelling

Separación de los dos datasets (train y test) por los años 2011-2013 para el train y 2014 para el test

In [ ]:
# Asegurarse de que las fechas estén ordenadas
data = data.sort_values('Order Date')

# Agregar ventas diarias
daily_sales = data.groupby('Order Date')['Sales'].sum().reset_index()

# Dividir en train y test
train = daily_sales[(daily_sales['Order Date'] >= '2011-01-01') & (daily_sales['Order Date'] <= '2013-12-31')]
test = daily_sales[(daily_sales['Order Date'] >= '2014-01-01') & (daily_sales['Order Date'] <= '2014-12-31')]

print("Train:", train['Order Date'].min(), "a", train['Order Date'].max())
print("Test:", test['Order Date'].min(), "a", test['Order Date'].max())


Exploracion visual de los datos (train)

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Configuración de índice temporal
train.set_index('Order Date', inplace=True)

# Descomposición de la serie
decompose_result = seasonal_decompose(train['Sales'], model='additive', period=365)

# Gráficos de descomposición
decompose_result.plot()
plt.show()

# Visualización adicional de la tendencia
plt.figure(figsize=(10, 5))
plt.plot(train['Sales'], label='Ventas Diarias')
plt.plot(decompose_result.trend, label='Tendencia', color='orange')
plt.legend()
plt.show()


Prueba de test AdFuller o ADF para detectar estacionalidad en los datos tanto de train como de test

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(series, title=""):
    result = adfuller(series.dropna())
    print(f'{title} ADF Test Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:', result[4])

# Prueba para train y test
test_stationarity(train['Sales'], "Train")
test_stationarity(test.set_index('Order Date')['Sales'], "Test")


Transformamos los datos de test viendo los resultados que no son estacionarios

In [ ]:
test['Sales_diff'] = test['Sales'] - test['Sales'].shift(1)
from statsmodels.tsa.stattools import adfuller
test_stationarity(test['Sales_diff'].dropna(), "Test Diferenciado")
train['Sales_diff'] = train['Sales'] - train['Sales'].shift(1)
from statsmodels.tsa.stattools import adfuller
test_stationarity(train['Sales_diff'].dropna(), "Train Diferenciado")

Lanzamos los modelos a entrenar en el día a día. Usaremos SARIMA y ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Modelo ARIMA en datos diferenciados de Train
arima_model = ARIMA(train['Sales_diff'].dropna(), order=(1, 1, 1)).fit()

# Predicciones en Test (diferenciado)
arima_forecast_diff = arima_model.forecast(steps=len(test['Sales_diff'].dropna()))

# Reintegración de las predicciones a la escala original
arima_forecast = arima_forecast_diff.cumsum() + train['Sales'].iloc[-1]


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Modelo SARIMA en datos diferenciados de Train
sarima_model = SARIMAX(train['Sales_diff'].dropna(), 
                       order=(1, 1, 1), 
                       seasonal_order=(1, 1, 1, 365)).fit()

# Predicciones en Test (diferenciado)
sarima_forecast_diff = sarima_model.forecast(steps=len(test['Sales_diff'].dropna()))

# Reintegración de las predicciones a la escala original
sarima_forecast = sarima_forecast_diff.cumsum() + train['Sales'].iloc[-1]


Analizamos el funcionamiento del modelo  con las diferentes métricas de RMSE y MAE

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Evaluación para ARIMA
arima_rmse = np.sqrt(mean_squared_error(test['Sales'], arima_forecast))
arima_mae = mean_absolute_error(test['Sales'], arima_forecast)

# Evaluación para SARIMA
sarima_rmse = np.sqrt(mean_squared_error(test['Sales'], sarima_forecast))
sarima_mae = mean_absolute_error(test['Sales'], sarima_forecast)

print("ARIMA - RMSE:", arima_rmse, "MAE:", arima_mae)
print("SARIMA - RMSE:", sarima_rmse, "MAE:", sarima_mae)


Planteamos la predicción de 2015 y la enfrentamos a los datos de 2014 tanto reales como predecidos

In [ ]:
# Predicciones para 2015 (365 días)
sarima_2015_forecast_diff = sarima_model.get_forecast(steps=365).predicted_mean

# Reintegrar predicciones a la escala original
sarima_2015_forecast = sarima_2015_forecast_diff.cumsum() + train['Sales'].iloc[-1]

# Visualización de las predicciones
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(test['Order Date'], test['Sales'], label='Datos Reales (2014)')
plt.plot(test['Order Date'], sarima_forecast, label='Predicciones (2014)')
plt.plot(pd.date_range('2015-01-01', periods=365), sarima_2015_forecast, label='Predicciones (2015)', color='orange')
plt.legend()
plt.show()

Buscamos la mejora de los parámetros con pdarima

In [ ]:
from pmdarima import auto_arima

# GridSearch para SARIMA
auto_model = auto_arima(train['Sales_diff'].dropna(), 
                        seasonal=True, m=365, 
                        trace=True, error_action='ignore', 
                        suppress_warnings=True, stepwise=True)

print("Mejores parámetros SARIMA:")
print("Order:", auto_model.order)
print("Seasonal Order:", auto_model.seasonal_order)


Reentrenamos el modelo con los parámetros sacados del pmdarima

In [ ]:
# Retrain con los mejores parámetros encontrados
best_sarima_model = SARIMAX(train['Sales_diff'].dropna(),
                            order=auto_model.order, 
                            seasonal_order=auto_model.seasonal_order).fit()

# Predicciones en Test
best_sarima_forecast_diff = best_sarima_model.forecast(steps=len(test['Sales_diff'].dropna()))
best_sarima_forecast = best_sarima_forecast_diff.cumsum() + train['Sales'].iloc[-1]


Volvemos a reevaluar el modelo SARIMA para ver si han mejorado las métricas

In [ ]:
# Evaluación del modelo optimizado
best_sarima_rmse = np.sqrt(mean_squared_error(test['Sales'], best_sarima_forecast))
best_sarima_mae = mean_absolute_error(test['Sales'], best_sarima_forecast)

print("SARIMA Optimizado - RMSE:", best_sarima_rmse, "MAE:", best_sarima_mae)


Una vez aquí, planteamos un cambio en el modelado y volvemos a hacer los mismos pasos pero de forma semanal

Cargamos datos de nuevo

In [ ]:
import pandas as pd

# Cargar el archivo
file_path = './data/orders_orders.xlsx'
df = pd.read_excel(file_path)

# Ver rango de fechas
print("Rango de fechas:", df['Order Date'].min(), "a", df['Order Date'].max())

Aqui hacemos el sumatorio semanal y nos aseguramos sobre todo que haya el mismo numero de semanas

In [ ]:
# Agregar la columna 'Week' basada en la fecha
df['Year'] = df['Order Date'].dt.year
df['Week'] = df['Order Date'].dt.isocalendar().week

# Resumir las ventas por semana
df_weekly = df.groupby(['Year', 'Week'])['Sales'].sum().reset_index()

# Separar en train (2011-2013) y test (2014)
train = df_weekly[df_weekly['Year'] < 2014]
test = df_weekly[df_weekly['Year'] == 2014]

In [ ]:
# Crear una lista con todas las combinaciones de año y semana
all_weeks = [(year, week) for year in range(2011, 2015) for week in range(1, 53)]

# Crear un DataFrame con todas las combinaciones posibles de año y semana
weeks_df = pd.DataFrame(all_weeks, columns=['Year', 'Week'])

# Unir con los datos agregados, completando los valores faltantes
df_weekly_full = pd.merge(weeks_df, df_weekly, on=['Year', 'Week'], how='left').fillna(0)

Ofrecemos la visualización de los datos ahora ya partidos de forma semanal

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd

# Crear la columna 'Date' con el primer día de cada semana
# Para esto, concatenamos el año y la semana y usamos pd.to_datetime con formato adecuado
df_weekly_full['Date'] = pd.to_datetime(df_weekly_full['Year'].astype(str) + df_weekly_full['Week'].astype(str) + '1', format='%Y%W%w')

# Establecer 'Date' como índice
df_weekly_full.set_index('Date', inplace=True)

# Descomposición de la serie temporal (estacionalidad, tendencia)
decomposition = sm.tsa.seasonal_decompose(df_weekly_full['Sales'], model='additive', period=52)
decomposition.plot()
plt.show()

# Visualización de la serie temporal completa
df_weekly_full['Sales'].plot(figsize=(10,6))
plt.title("Ventas Semanales")
plt.xlabel("Fecha")
plt.ylabel("Ventas")
plt.show()





Lanzamos ahora de nuevo el test de ADF ya sabiendo que como los datos de 2014 pueden salir no estacionarios los diferenciamos para que ambos sean estacionarios

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Función para comprobar estacionariedad
def check_stationarity(series):
    result = adfuller(series)
    return result[0], result[1]

# Realizar diferenciación si es necesario y comprobar la estacionariedad
# Differenciación de primer orden para entrenamiento
train_diff = train['Sales'].diff().dropna()

# Test ADF para el conjunto de train después de diferenciación
adf_train = check_stationarity(train_diff)
print(f"ADF Test Train (diferenciado): estadístico={adf_train[0]}, p-valor={adf_train[1]}")

# Differenciación de primer orden para test
test_diff = test['Sales'].diff().dropna()

# Test ADF para el conjunto de test después de diferenciación
adf_test = check_stationarity(test_diff)
print(f"ADF Test Test (diferenciado): estadístico={adf_test[0]}, p-valor={adf_test[1]}")



Entrenamos ahora los modelos pero ya con el formato de cifra semanal

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Función para comprobar estacionariedad
def check_stationarity(series):
    result = adfuller(series)
    return result[0], result[1]

# Realizar diferenciación si es necesario y comprobar la estacionariedad
# Differenciación de primer orden para entrenamiento
train_diff = train['Sales'].diff().dropna()

# Test ADF para el conjunto de train después de diferenciación
adf_train = check_stationarity(train_diff)
print(f"ADF Test Train (diferenciado): estadístico={adf_train[0]}, p-valor={adf_train[1]}")

# Differenciación de primer orden para test
test_diff = test['Sales'].diff().dropna()

# Test ADF para el conjunto de test después de diferenciación
adf_test = check_stationarity(test_diff)
print(f"ADF Test Test (diferenciado): estadístico={adf_test[0]}, p-valor={adf_test[1]}")



Analizamos los resultados que nos da el modelo con las métricas de error RMSE y MAE

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# RMSE y MAE para ARIMA
rmse_arima = np.sqrt(mean_squared_error(test['Sales'], pred_arima))
mae_arima = mean_absolute_error(test['Sales'], pred_arima)

# RMSE y MAE para SARIMA
rmse_sarima = np.sqrt(mean_squared_error(test['Sales'], pred_sarima))
mae_sarima = mean_absolute_error(test['Sales'], pred_sarima)

print(f"ARIMA RMSE: {rmse_arima}, MAE: {mae_arima}")
print(f"SARIMA RMSE: {rmse_sarima}, MAE: {mae_sarima}")

Lanzamos predicción del año 2015

In [ ]:
# Predicción para 2015 con el mejor modelo (por ejemplo, SARIMA)
pred_2015 = model_sarima_fit.forecast(steps=52)  # Predicción para 52 semanas


Visualizamos las diferentes curvas de ventaas semanales en 2014 real vs 2014 predecido

In [ ]:
import matplotlib.pyplot as plt

# Asegurarnos de que las predicciones están alineadas con las fechas del conjunto de prueba
pred_arima.index = test.index
pred_sarima.index = test.index

# Gráfico de comparación
plt.figure(figsize=(14, 8))

# Ventas reales
plt.plot(test.index, test['Sales'], label='Ventas Reales (2014)', color='blue')

# Predicciones ARIMA
plt.plot(test.index, pred_arima, label='Predicciones ARIMA', color='green', linestyle='--')

# Predicciones SARIMA
plt.plot(test.index, pred_sarima, label='Predicciones SARIMA', color='red', linestyle=':')

# Detalles del gráfico
plt.title("Comparación de Ventas Reales y Predicciones ARIMA/SARIMA", fontsize=16)
plt.xlabel("Fecha", fontsize=14)
plt.ylabel("Ventas Semanales", fontsize=14)
plt.legend(fontsize=12)
plt.grid()
plt.show()


Buscamos ver como predice los datos en 2015 visualmente vs 2014 real y el del modelo SARIMA

In [ ]:
date_range_2014 = pd.date_range(start='2014-01-01', end='2014-12-31', freq='W')
date_range_2015 = pd.date_range(start='2015-01-01', end='2015-12-31', freq='W')  # Ajustar el rango de fechas

# Asignar el índice de fechas al conjunto de predicciones de 2015
test.index = date_range_2014
pred_2015.index = date_range_2015
# Gráfico de comparación
plt.figure(figsize=(14, 8))

# Ventas reales (2014)
plt.plot(test.index, test['Sales'], label='Ventas Reales (2014)', color='blue')

# Predicciones ARIMA (2014)
plt.plot(test.index, pred_arima, label='Predicciones ARIMA (2014)', color='green', linestyle='--')

# Predicciones SARIMA (2014)
plt.plot(test.index, pred_sarima, label='Predicciones SARIMA (2014)', color='red', linestyle=':')

# Predicciones SARIMA (2015)
plt.plot(pred_2015.index, pred_2015, label='Predicciones SARIMA (2015)', color='orange', linestyle='-.')

# Detalles del gráfico
plt.title("Comparación de Ventas Reales (2014), Predicciones ARIMA/SARIMA y Predicciones para 2015", fontsize=16)
plt.xlabel("Fecha", fontsize=14)
plt.ylabel("Ventas Semanales", fontsize=14)
plt.legend(fontsize=12)
plt.grid()

# Ajustar el formato del eje x para mostrar 2014 y 2015
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45, fontsize=12)
plt.show()


Buscamos ver cuales son los mejores parámetros para el modelo semanal de SARIMA realizado más arriba

In [ ]:
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# Definir el espacio de parámetros
p = d = q = range(0, 3)  # Orden para (p,d,q)
P = D = Q = range(0, 2)  # Orden para (P,D,Q)
seasonal_period = [52]  # Periodo estacional

# Generar todas las combinaciones posibles de parámetros
param_combinations = list(itertools.product(p, d, q))
seasonal_combinations = list(itertools.product(P, D, Q, seasonal_period))

# Inicializar variables para almacenar el mejor modelo
best_aic = float('inf')
best_params = None
best_model = None

# Búsqueda de hiperparámetros
for param in param_combinations:
    for seasonal_param in seasonal_combinations:
        try:
            # Entrenar el modelo SARIMAX con los parámetros actuales
            model = SARIMAX(
                train['Sales'],
                order=param,
                seasonal_order=seasonal_param,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            model_fit = model.fit(disp=False)
            
            # Evaluar el modelo usando AIC
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_params = (param, seasonal_param)
                best_model = model_fit
        except Exception as e:
            continue

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros:", best_params)
print("Mejor AIC:", best_aic)

# Predecir usando el mejor modelo
pred = best_model.forecast(steps=len(test))


Volvemos a reentrenar el modelo de SARIMAX con los mejores parámetros conseguido arriba y visualizar el modelo como actua con las predicciones del test y del 2015

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error





# Limitar el conjunto de datos de 2011-2013 para el entrenamiento
train_filtered = train[(train['Year'] >= 2011) & (train['Year'] <= 2013)]
# Limitar el conjunto de datos de 2014-2015 para la prueba
test_filtered = test[(test['Year'] >= 2014) & (test['Year'] <= 2015)]

# Reentrenar el modelo SARIMAX con los mejores parámetros obtenidos
best_param, best_seasonal_param = best_params  # Usar los mejores parámetros de la búsqueda anterior

model_sarima_final = SARIMAX(
    train_filtered['Sales'],
    order=best_param,
    seasonal_order=best_seasonal_param,
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Ajustar el modelo
model_sarima_fit_final = model_sarima_final.fit()

# Predicciones para 2014 y 2015
pred_sarima_final = model_sarima_fit_final.forecast(steps=len(test_filtered))

# Evaluar el modelo con el RMSE o MAE
rmse = np.sqrt(mean_squared_error(test_filtered['Sales'], pred_sarima_final))
print(f'RMSE para los datos de 2014 y 2015: {rmse}')




Pintamos de nuevo el modelo como queda con 2014 real y predicción

In [ ]:
import matplotlib.pyplot as plt

# Visualizar las predicciones vs los datos reales para 2014 y 2015
plt.figure(figsize=(10, 6))
plt.plot(test_filtered.index, test_filtered['Sales'], label='Ventas reales', color='blue')
plt.plot(test_filtered.index, pred_sarima_final, label='Predicciones SARIMAX', color='red', linestyle='--')
plt.title('Comparación de Predicciones y Ventas Reales (2014-2015)')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.grid(True)
plt.show()

Mostramos el dato de 2015 comparado con el 2014 real y predecido

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Crear el rango de fechas para 2014 y 2015 (semanal)
date_range_2014 = pd.date_range(start='2014-01-01', end='2014-12-31', freq='W')
date_range_2015 = pd.date_range(start='2015-01-01', end='2015-12-31', freq='W')

# Asignar el índice de fechas al conjunto de predicciones de 2015
test_filtered.index = date_range_2014
pred_sarima_final.index = date_range_2015  # Asegurarse que las predicciones tienen el índice de 2015

# Gráfico de comparación
plt.figure(figsize=(14, 8))

# Ventas reales (2014 y 2015)
plt.plot(test_filtered.index, test_filtered['Sales'], label='Ventas Reales (2014-2015)', color='blue')

# Predicciones SARIMA (2014)
plt.plot(test_filtered.index, pred_sarima_final[:len(test_filtered)], label='Predicciones SARIMA (2014)', color='red', linestyle=':')

# Predicciones SARIMA (2015)
plt.plot(pred_sarima_final.index, pred_sarima_final, label='Predicciones SARIMA (2015)', color='orange', linestyle='-.')

# Detalles del gráfico
plt.title("Comparación de Ventas Reales (2014-2015) y Predicciones SARIMA para 2014 y 2015", fontsize=16)
plt.xlabel("Fecha", fontsize=14)
plt.ylabel("Ventas Semanales", fontsize=14)
plt.legend(fontsize=12)
plt.grid()

# Ajustar el formato del eje x para mostrar las fechas correctamente
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45, fontsize=12)

plt.show()
